In [1]:
%matplotlib tk

import argparse
import gym
import datetime
import os
import random
import tempfile
import numpy as np
import pickle

import ray
from ray import tune
from ray.tune.logger import Logger, UnifiedLogger, pretty_print
from ray.rllib.env.multi_agent_env import make_multi_agent
from ray.rllib.examples.models.shared_weights_model import TF2SharedWeightsModel
from ray.rllib.models import ModelCatalog
from ray.rllib.utils.framework import try_import_tf
from ray.rllib.utils.test_utils import check_learning_achieved
from ray.rllib.agents.ppo import ppo, PPOTrainer, PPOTFPolicy
from ray.rllib.agents.a3c.a3c_tf_policy import A3CTFPolicy
from ray.rllib.agents.a3c import a3c
from ray.rllib.models import ModelCatalog
from ray.rllib.policy.policy import PolicySpec
from environment_rllib_3d import MyEnv
from settings.initial_settings import *
from settings.reset_conditions import reset_conditions
#from modules.models import MyConv2DModel_v0B_Small_CBAM_1DConv_Share
from modules.models import DenseNetModelLarge
from tensorflow.keras.utils import plot_model
from modules.savers import save_conditions
from utility.result_env import render_env
from utility.terminate_uavsimproc import teminate_proc
from utility.latest_learned_file_path import latest_learned_file_path
from utility.save_logs import save_logs

import matplotlib.pyplot as plt
import matplotlib
import tensorflow as tf
import cv2
import ctypes
import warnings

#UCAV.exeが起動している場合、プロセスキルする。
teminate_proc.UAVsimprockill(proc_name="UCAV.exe")

warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 
warnings.filterwarnings('ignore', category=matplotlib.MatplotlibDeprecationWarning)
np.set_printoptions(precision=3, suppress=True)
PROJECT = "UCAV"
TRIAL_ID = 2
TRIAL = 'test_' + str(TRIAL_ID)
EVAL_FREQ = 10
CONTINUAL = False
NUM_EVAL = 1
def custom_log_creator(custom_path, custom_str):
    timestr = datetime.datetime.today().strftime("%Y-%m-%d_%H-%M-%S")
    logdir_prefix = "{}_{}".format(custom_str, timestr)

    def logger_creator(config):
        if not os.path.exists(custom_path):
            os.makedirs(custom_path)
        logdir = tempfile.mkdtemp(prefix=logdir_prefix, dir=custom_path)
        return UnifiedLogger(config, logdir, loggers=None)

    return logger_creator

ray.shutdown()
ray.init(ignore_reinit_error=True, log_to_driver=False)

ModelCatalog.register_custom_model('my_model', DenseNetModelLarge)

# config = {"env": MyEnv,
#           "num_workers": NUM_WORKERS,
#           "num_gpus": NUM_GPUS,
#           "num_cpus_per_worker": NUM_CPUS_PER_WORKER,
#           "num_sgd_iter": NUM_SGD_ITER,
#           "lr": LEARNING_RATE,
#           "gamma": GAMMA,  # default=0.99
#           "model": {"custom_model": "my_model"}
#           # "framework": framework
#           }  # use tensorflow 2
eval_env = MyEnv({})
policies = {
    #"blue_1": PolicySpec(config={"gamma": 0.99}),
    #"blue_2": PolicySpec(config={"gamma": 0.95}),
    "blue_0": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    "blue_1": (PPOTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_0": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
    #"blue_1": (A3CTFPolicy, eval_env.observation_space, eval_env.action_space, {}),
}
policy_ids = list(policies.keys())

def policy_mapping_fn(agent_id, episode, **kwargs):
    #print(agent_id,episode)
    #pol_id = policy_ids[agent_id]

    pol_id = agent_id
    return pol_id

# Instanciate the evaluation env

config = {"env": MyEnv,"num_gpus": 0,"num_workers": 0, "num_cpus_per_worker": 0,"num_gpus_per_worker": 0,
          "create_env_on_driver": True,"train_batch_size": 600*2*5,"batch_mode": "complete_episodes",
          "shuffle_sequences": True, "gamma":0.99, "lr": 1e-4,
          "clip_actions":True,"normalize_actions":True,
          "observation_space":eval_env.observation_space,"action_space":eval_env.action_space,
          "explore":True,
          "sgd_minibatch_size": 200, "num_sgd_iter":10,
          "exploration_config": {"type": "StochasticSampling","random_timesteps":0}, #PPO デフォルト
          "model":{"fcnet_activation": "tanh","fcnet_hiddens": [64, 64],"post_fcnet_activation": "linear",
                  "vf_share_layers": True,},#"linear","relu","tanh" "use_lstm":True,"lstm_cell_size":256,"max_seq_len":128
          #"model":{"fcnet_hiddens": [256, 256]},
          "multiagent": {"policies": policies,  "policy_mapping_fn": policy_mapping_fn}
         }
res_name = "sgd"+str(config["sgd_minibatch_size"])+"sgd_num"+str(config["num_sgd_iter"])+"lr"+str(config["lr"])+"gamma"+str(config["gamma"])
res_name = "test"
conditions_dir = os.path.join('./' + PROJECT + '/conditions/')

if not os.path.exists(conditions_dir):
    os.makedirs(conditions_dir)
save_conditions(conditions_dir)

# PPOTrainer()は、try_import_tfを使うと、なぜかTensorflowのeager modeのエラーになる。

trainer = ppo.PPOTrainer(config=config,
                         logger_creator=custom_log_creator(
                             os.path.expanduser("./" + PROJECT + "/logs"), TRIAL))

if CONTINUAL:
    # Continual learning: Need to specify the checkpoint
    # model_path = PROJECT + '/checkpoints/' + TRIAL + '/checkpoint_000197/checkpoint-197'
    model_path = latest_learned_file_path('./UCAV/checkpoints/test_2/*')
    trainer.restore(checkpoint_path=model_path)

# models_dir = os.path.join('./' + PROJECT + '/models/')
# if not os.path.exists(models_dir):
#     os.makedirs(models_dir)
# text_name = models_dir + TRIAL + '.txt'
# with open(text_name, "w") as fp:
#     trainer.get_policy().model.base_model.summary(print_fn=lambda x: fp.write(x + "\r\n"))
# png_name = models_dir + TRIAL + '.png'
# plot_model(trainer.get_policy().model.base_model, to_file=png_name, show_shapes=True)



# Define checkpoint dir
check_point_dir = os.path.join('./' + PROJECT + '/checkpoints/', TRIAL)
if not os.path.exists(check_point_dir):
    os.makedirs(check_point_dir)

C:\Users\Takumi\anaconda3\envs\AI2\lib\site-packages\quaternion\numba_wrapper.py:23: UserWarning: 

!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Could not import from numba, which means that some
parts of this code may run MUCH more slowly.  You
may wish to install numba.
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

  warnings.warn(warning_text)
2022-01-12 01:49:04,181	INFO trainer.py:723 -- Your framework setting is 'tf', meaning you are using static-graph mode. Set framework='tf2' to enable eager execution with tf2.x. You may also want to then set `eager_tracing=True` in order to reach similar execution speed as with static-graph mode.
2022-01-12 01:49:04,181	INFO ppo.py:167 -- In multi-agent mode, policies will be optimized sequentially by the multi-GPU optimizer. Consider setting simple_optimizer=True if this doesn't work for you.
2022-01-12 01:49:04,182	INFO trainer.py:745 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or u

In [ ]:
record_mode = 0
results_dir = os.path.join('./' + PROJECT + '/results/')

if not os.path.exists(results_dir):
    os.makedirs(results_dir)
results_file = results_dir + TRIAL + '.pkl'
for steps in range(10001):
    # Training
    print(f'\n----------------- Training at steps:{steps} start! -----------------')
    eval_env.reset()
    results = trainer.train()
    save_logs(res_name,results,steps,CONTINUAL)
    print(pretty_print(results))
    #check_point = trainer.save(checkpoint_dir=check_point_dir)
    # Evaluation
    if steps % EVAL_FREQ == 0:
        print(f'\n----------------- Evaluation at steps:{steps} starting ! -----------------')
        #print(pretty_print(results))
        check_point = trainer.save(checkpoint_dir=check_point_dir)
        for i in range(NUM_EVAL):
            # print(f'\nEvaluation {i}:')
            obs = eval_env.reset()
            done = False
            
            step_num = 0
            fig = plt.figure(1,figsize=(8.0, 6.0))
            ESC = 0x1B          # ESCキーの仮想キーコード
            trajectory_length = 100
            env_blue_pos = [0]
            env_red_pos = [0]
            env_mrm_pos = [0]
            if record_mode == 0:
                file_name = "test_num" + str(steps) +str(i)
                video = cv2.VideoWriter(file_name+'.mp4',0x00000020,20.0,(800,600))

            while True:
                action_dict = {}
                for j in range(eval_env.blue_num):
                    #if not eval_env.blue[j].hitpoint == 0:
                    #action_dict['blue_' + str(j)] = trainer.compute_action(obs['blue_' + str(j)])
                    action_dict['blue_' + str(j)] = trainer.compute_single_action(obs['blue_' + str(j)],policy_id='blue_' + str(j),
                                                                       clip_actions=True,explore=False)
                obs, rewards, dones, infos = eval_env.step(action_dict)
                env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp= render_env.copy_from_env(eval_env)
                env_blue_pos.append(env_blue_pos_temp)
                env_red_pos.append(env_red_pos_temp)
                env_mrm_pos.append(env_mrm_pos_temp)
                if step_num == 0:
                    del env_blue_pos[0]
                    del env_red_pos[0]
                    del env_mrm_pos[0]

                hist_blue_pos = np.vstack(env_blue_pos)
                hist_red_pos = np.vstack(env_red_pos)
                hist_mrm_pos = np.vstack(env_mrm_pos)
                plt.clf()
                render_env.rend_3d(eval_env,hist_blue_pos,"b",1)
                render_env.rend_3d(eval_env,hist_red_pos,"r",1)
                render_env.rend_3d(eval_env,hist_mrm_pos,"k",1)
                plt.subplots_adjust(left=-0.5, right=1.5, bottom=-0.25, top=1.25)
                fig.canvas.draw()
                plt.pause(.01)
                if record_mode == 0:
                    img = np.array(fig.canvas.renderer.buffer_rgba())
                    img = cv2.cvtColor(img, cv2.COLOR_RGBA2BGR)
                    video.write(img.astype('uint8'))

                
                step_num = step_num + 1
                
                done = dones["__all__"]
                #print(f'rewards:{rewards}')
                #if record_mode == 0:
                #    img = eval_env.render_movie(file_name,step_num)
                #    video.write(img.astype('unit8'))
                #elif record_mode == 1:
                #    eval_env.render()
                #elif record_mode == 2:
                #    eval_env.render()
                    
                #env_blue_pos_temp, env_red_pos_temp, env_mrm_pos_temp = render_env.copy_from_env(eval_env)
                
                #env_blue_pos.append(env_blue_pos_temp)
                #env_red_pos.append(env_red_pos_temp)
                #env_mrm_pos.append(env_mrm_pos_temp)
                #step_num = step_num + 1
                # エピソードの終了処理
                if dones['__all__']:
                    # print(f'all done at {env.steps}')
                    break
                
            #del env_blue_pos[0]
            #del env_red_pos[0]
            #del env_mrm_pos[0]
            
            #hist_blue_pos = np.vstack(env_blue_pos)
            #hist_red_pos = np.vstack(env_red_pos)
            #hist_mrm_pos = np.vstack(env_mrm_pos)
            
            #f = open(results_file,'wb')
            #pickle.dump(emv_blue_pos,f)
            #pickle.dump(emv_red_pos,f)
            #pickle.dump(emv_mrm_pos,f)
            #f.close()
            
            if record_mode == 0:
                video.release()

ray.shutdown()


----------------- Training at steps:0 start! -----------------
343 red_1 Shoot at blue_1
358 blue_1 Shoot at red_1 launch distance : 35667.148400173515 True True
Same tgt shoot
359 blue_1 Shoot at red_1 launch distance : 35358.15162024169 True True
407 red_0 Shoot at blue_1
410 red_1 Splash :blue_1
540 blue_0 Shoot at red_0 launch distance : 35104.10957423646 True True
Same tgt shoot
541 blue_0 Shoot at red_0 launch distance : 34726.4966992065 True True
598 blue_0 DOWN
DOWN LOSE
blue_0 True False 598 -10.002 -15.191999999999965
blue_1 True False 598 -0.002 -1.2300000000000009
368 red_0 Shoot at blue_1
425 red_1 Shoot at blue_0
426 blue_1 Shoot at red_0 launch distance : 30977.306806757748 True True
438 red_0 Splash :blue_1
477 blue_0 Shoot at red_1 launch distance : 31935.340721526678 True True
Same tgt shoot
478 blue_0 Shoot at red_1 launch distance : 31594.663441790293 True True
481 blue_1 Splash :red_0
494 red_1 Splash :blue_0
TIME LIMIT LOSE
blue_0 False True 494 -5.002 -9.6280000

2022-01-12 01:59:26,766	WARNING deprecation.py:46 -- DeprecationWarning: `slice` has been deprecated. Use `SampleBatch[start:stop]` instead. This will raise an error in the future!


agent_timesteps_total: 13196
custom_metrics: {}
date: 2022-01-12_01-59-28
done: false
episode_len_mean: 599.8181818181819
episode_media: {}
episode_reward_max: -2.0619999999999497
episode_reward_mean: -17.309818181818166
episode_reward_min: -22.24399999999996
episodes_this_iter: 11
episodes_total: 11
experiment_id: b8082c9356f1449d82800d09b5f2e7c4
hostname: DESKTOP
info:
  learner:
    blue_0:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 9.999999747378752e-05
        entropy: 7.148765563964844
        entropy_coeff: 0.0
        kl: 0.005915842019021511
        model: {}
        policy_loss: 0.03075597621500492
        total_loss: 4.2151384353637695
        vf_explained_var: -0.03796883672475815
        vf_loss: 4.183199405670166
    blue_1:
      custom_metrics: {}
      learner_stats:
        cur_kl_coeff: 0.20000000298023224
        cur_lr: 9.999999747378752e-05
        entropy: 7.148980140686035
        entropy_coeff: 0.0
  

2022-01-12 01:59:32,270	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:32,312	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:32,964	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:32,966	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2022-01-12 01:59:38,554	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:38,997	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:39,000	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 01:59:39,468	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


...

2022-01-12 02:04:41,118	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:41,120	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:41,434	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:41,436	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2022-01-12 02:04:46,439	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:46,767	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:46,770	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:47,088	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2022-01-12 02:04:52,179	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:52,181	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:52,502	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:52,505	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


2022-01-12 02:04:57,505	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:57,815	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:57,818	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!
2022-01-12 02:04:58,145	WARNING deprecation.py:46 -- DeprecationWarning: `Trainer.compute_single_action(`clip_actions`=...)` has been deprecated. Use `Trainer.compute_single_action(`clip_action`=...)` instead. This will raise an error in the future!


828 red_0 Splash :blue_1
TIME LIMIT LOSE
blue_0 False True 828 -5.002 -6.248000000000001
blue_1 False True 828 -5.002 -11.185999999999886

----------------- Training at steps:1 start! -----------------
422 red_1 Shoot at blue_1
430 red_0 Shoot at blue_1
434 blue_1 Shoot at red_0 launch distance : 38239.06595355069 True True
Same tgt shoot
435 blue_1 Shoot at red_0 launch distance : 37838.164622507786 True True
476 red_1 Splash :blue_1
629 red_1 Shoot at blue_0
646 blue_0 Shoot at red_1 launch distance : 35252.37400516453 True True
Same tgt shoot
648 blue_0 Shoot at red_1 launch distance : 34747.06679706936 True True
685 red_1 Splash :blue_0
TIME LIMIT LOSE
blue_0 False True 685 -5.002 -10.521999999999961
blue_1 False True 685 -5.002 -6.766000000000001
434 red_0 Shoot at blue_1
473 red_1 Shoot at blue_1
477 blue_1 Shoot at red_0 launch distance : 31850.457798907693 True True
Same tgt shoot
478 blue_1 Shoot at red_0 launch distance : 31718.997288691204 True True
498 red_0 Splash :blue_1
